<a href="https://colab.research.google.com/github/jlmcc94303/A-practice-article/blob/master/TwoHiddenLayerMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Neural Network Tutorial
for use with *Deep Learning*, by J. L. McClelland & M. Botvinick.

To appear in M. Kahana & A. Wagner (Eds), *Oxford Handbook of Human Memory*, 2020, OUP.

Based on 'Tensorflow quickstart for beginners' 
by The Tensorflow Authors. Adapted by J. McClelland.

##### Copyright 2019 The TensorFlow Authors,
Adaptation Copyright 2020 James L, McClelland

For the original TF quickstart for beginners, follow these links:

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.
4. Examine the output confusability matrix and
hidden-layer representational similarity matrix
after training

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Note: To rerun after making changes, it is best to select *Runtime* > *Restart runtime* before selecting *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: It may be necessary to upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [0]:
import tensorflow as tf
import numpy as np; import numpy.matlib; import matplotlib.pyplot as plt
%matplotlib inline
import math; import scipy as sp; import scipy.linalg

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` network by stacking layers. We have broken the network into parts
to make it easy to access the patterns of activation in the hidden layers of the network.

The original tutorial includes Dropout, which you can read about [on this blog](https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/).  To see what happens, uncomment the line inserting Dropout in the code block below.

In [0]:
hidden1 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu')
])

hidden2 = tf.keras.models.Sequential([
  hidden1, tf.keras.layers.Dense(128, activation='relu')
])

model = tf.keras.models.Sequential([
  hidden2, #tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

Let's look at what the model produces for one example.

In [0]:

predictions = model(x_train[:1]).numpy()
predictions

The `tf.nn.softmax` function converts these logits to "probabilities" for each class.  Think of these as estimates of the probability that the input is a given digit.  These numbers must sum to 1. Note that the numbers can be of the form D.DD...e-N where N is a small integer that indicates the number of places to shift the decimal to the left so that 8.17...e-1 = .871..., for example.

In [0]:
tf.nn.softmax(predictions).numpy()

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and the index of the `True` label and returns a scalar loss for each example.

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn(y_train[:1], predictions).numpy()

Now, we are going to compile the model using Keras, and we will tell it what optimizer to use.  We use Stocastic Gradient Descent (SGD), though Adam is more stable and works faster in this case.  See [this blog](https://ruder.io/optimizing-gradient-descent/) for a discussion.

In [0]:
model.compile(optimizer='SGD', # or try 'adam' instead
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss.  You can specify the number of epochs to train, where an epoch is a sweep through the whole training set, divided into batches presented in random order.  If you train for only 1 epoch, the performance will not be great if you use SGD.  5 epochs with adam is enough for very good performance.  You can re-run this code block to train for more epochs (provided you haven't changed anything above!)

In [0]:
model.fit(x_train, y_train, epochs=1)

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)", although the internal name of the set being used is 'test', not 'validation'.  It is a set of 10,000 items not used in training.

In [0]:
model.evaluate(x_test,  y_test, verbose=2)

After 5 epochs with the adam optimizer, the network achieves close to ~98% accuracy on this validation set. We don't do as well with SGD, especially not after just one epoch.  To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [0]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

Now we are going to look at the network's output probabilities.  We average the probability vector for each of the digits 0 through 9, then plot these
in a matrix, which shows that on average, the network thinks that the correct digit is more likely.

After just one epoch with SGD, the network is generally assigning most of the propobility to the correct digit, though 3's and 5's and 4's and 9's tend to have some probability mass on each other.  To allow small probability values to be visible in the display, we have taken each probability to the .5 power before displaying.  The largest actual off-diagonal probability is printed above the display, and we have shown the actual probabilities corresponding to the displayed colors on the colorbar.



In [0]:
predmeans = np.zeros((10,10))
prv = .5 # scales values for display to make small values visually distinct
outputs = probability_model(x_test).numpy()
for yv in range (10):
  inds = np.where(y_test==yv)
  predmeans[yv,:] = np.mean(outputs[inds],axis=0)
tpm = np.copy(predmeans)
for yv in range(10): tpm[yv,yv] = 0.0
print('largest off-diagonal value is',np.amax(tpm))
spredmeans = np.power(predmeans,prv) # we apply the scaling here
fig, ax = plt.subplots()
cax = ax.imshow(spredmeans,cmap='Purples',vmin=0,vmax=1); 
cbar =fig.colorbar(cax,cmap='Purples',ticks = [0.0,.2,.4,.6,.8,1.0])
tks = np.arange(0,10);tt = plt.xticks(tks);tt = plt.yticks((tks));
# next line assumes prv == .5
cbar.ax.set_yticklabels(['0.00','0.04','0.16','0.36','0.64', '1.00'])
tt = ax.set_ylabel('Correct Digit',fontsize = 14)
tt = ax.set_xlabel('Predicted Probability',fontsize=14)

Next, we compute the representation similarity matrix of the patterns at the first hidden layer. We use Cosine similarity.  The Cosine ranges between +1 and -1, for patterns that are identical to those that are exactly numerically opposite.  Since the hidden layer pattern elements are all non-negative, the cosines all tend to be positive, and indeed, the smallest values are typically about .5, so we display them scaled between 1 and .5.

In [0]:
hidpats = np.zeros((10000,128),dtype=np.float32)
hidpats = hidden1(x_test).numpy()
hidmeans = np.zeros((10,128))
for yv in range (10):
  inds = np.where(y_test==yv)
  hidmeans[yv,:] = np.mean(hidpats[inds],axis=0)
cdm = sp.spatial.distance.pdist(hidmeans,'cosine')
rsm = 1 - sp.spatial.distance.squareform(cdm)
plt.imshow(rsm,cmap='Purples',vmin=.4,vmax=1)
plt.colorbar(cmap='Purples'); 
tt = plt.xticks(tks);tt = plt.yticks((tks)); 
tt = plt.xlabel('Digit',fontsize = 14)
tt = plt.ylabel('Digit',fontsize=14)

Finally, we do the same for the patterns at the second hidden layer.  You will see that the pattern is similar but the differences are somewhat more exaggerated.

In [0]:
hidpats = np.zeros((10000,128),dtype=np.float32)
hidpats = hidden2(x_test).numpy()  
hidmeans = np.zeros((10,128))
for yv in range (10):
  inds = np.where(y_test==yv)
  hidmeans[yv,:] = np.mean(hidpats[inds],axis=0)
cdm = sp.spatial.distance.pdist(hidmeans,'cosine')
rsm = 1 - sp.spatial.distance.squareform(cdm)
# above makes the matrix square and converts cosine distance to
# cosine similarity, producing a represenational similarity matrix
plt.imshow(rsm,cmap='Purples',vmin=.4,vmax=1)
plt.colorbar(cmap='Purples')
tt = plt.xticks(tks);tt = plt.yticks((tks)); 
tt = plt.xlabel('Digit',fontsize = 14)
tt = plt.ylabel('Digit',fontsize=14)

From here, you can go back and change things or just run more epochs to see how the results change.  

Note: Running model.fit for more epochs and then running the subsequent cells manually seems to work fine. If you make changes before the model.fit cell, things will be out of sync. You should restart your run time, then run the first cell of the notebook, then you can chose *Runtime* -> *Run after* to run the rest of the cells.